In [1]:
import os
import numpy as np
import json
import copy
import csv
import scipy
import matplotlib.pyplot as plt 
from statsmodels.regression.linear_model import OLS, WLS
from statsmodels.regression.mixed_linear_model import MixedLM
from statsmodels.tools.tools import add_constant

In [2]:
data_path = "../../../data/vi_data"
ori_vis = [f"{i}_{j}" for i in ["cataract", "amd", "glaucoma", "presbyopia", "myopia"] for j in [1, 2, 3]] + ["normal_1"]
tasks = ["Catch", "Navigate", "Place", "Slice", "Throw", "Wipe"]
ids = range(1, 11)
save_json = True

In [3]:
dv_1 = [
    "collision_freq",
    "task_completion_time",
    "throw_distance_from_ball_to_target",
    "num_collisions",
    "total_collision_time",
    "wipe_total_irrevalant_objects_disarrangement",
    "catch_failure_rate",
    "throw_failure_rate",
    "raw_failure_rate",
]

dv_2 = [
    "mean_pupil_dilation",
    "total_body_translation",
    "total_body_rotation",
    "total_fixation_count",
    "total_fixation_duration",
    "total_gaze_movement",
    "total_hmd_rotation",
    "total_hmd_translation",
    "total_left_hand_rotation",
    "total_left_hand_translation",
    "total_right_hand_rotation",
    "total_right_hand_translation",
]  

In [4]:
dv1_data, dv2_data, dr = {}, {}, {}
for dv in dv_1:
    with open(f"{data_path}/json/dv1/{dv}.json", "r") as f:
        dv1_data[dv] = json.load(f)
for dv in dv_2:
    with open(f"{data_path}/json/dv2/{dv}.json", "r") as f:
        dv2_data[dv] = json.load(f)

with open(f"{data_path}/json/subject_ratings.json", "r") as f:
    sr = json.load(f)

In [5]:
# rename key for catch and throw success rate
for vi in ori_vis:
    dv1_data["catch_failure_rate"][f"catch_{vi}"] = dv1_data["catch_failure_rate"].pop(vi)
    dv1_data["throw_failure_rate"][f"throw_{vi}"] = dv1_data["throw_failure_rate"].pop(vi)
    for id in ids:
        dv1_data["catch_failure_rate"][f"catch_{vi}"][str(id)] = {'0': dv1_data["catch_failure_rate"][f"catch_{vi}"][str(id)]}
        dv1_data["throw_failure_rate"][f"throw_{vi}"][str(id)] = {'0': dv1_data["throw_failure_rate"][f"throw_{vi}"][str(id)]}

In [6]:
post_processed_ratings = {task: [] for task in tasks}

vis = {
    "Glaucoma": [f"glaucoma_{i}" for i in range(1, 4)], 
    "AMD": [f"amd_{i}" for i in range(1, 4)], 
    "Cataract": [f"cataract_{i}" for i in range(1, 4)], 
    "Myopia": [f"myopia_{i}" for i in range(1, 4)], 
    "Presbyopia": [f"presbyopia_{i}" for i in range(1, 4)], 
    "Normal": ["normal_1"]
}

for task in tasks:
    for vi in vis:
        if vi != "Normal":
            post_processed_ratings[task].append(np.mean([sr[f"{task} [{vi}]"][str(id)] for id in ids]))
        else:
            post_processed_ratings[task].append(1)

### Investigating subject rating and 1st DVs

In [7]:
dv1_data.keys()

dict_keys(['collision_freq', 'task_completion_time', 'throw_distance_from_ball_to_target', 'num_collisions', 'total_collision_time', 'wipe_total_irrevalant_objects_disarrangement', 'catch_failure_rate', 'throw_failure_rate', 'raw_failure_rate'])

In [8]:
tasks_dv1 = {
    "Slice": "task_completion_time",
    "Throw": "raw_failure_rate",
    "Catch": "catch_failure_rate",
    "Navigate": "total_collision_time",
    "Place": "task_completion_time",
    "Wipe": "task_completion_time"
}

post_processed_dv1 = {task: [] for task in tasks_dv1}

for task in tasks_dv1:
    for vi in vis:
        if task == "Navigate":
            post_processed_dv1[task].append(np.mean(
                [np.mean([np.mean(
                    np.array(list(dv1_data[tasks_dv1[task]][f"{task.lower()}_{v}"][str(id)].values())) / np.array(list(dv1_data["task_completion_time"][f"{task.lower()}_{v}"][str(id)].values()))
                ) for id in ids])
                for v in vis[vi]])
            )
        else:
            post_processed_dv1[task].append(np.mean(
                [np.mean([np.mean(list(dv1_data[tasks_dv1[task]][f"{task.lower()}_{v}"][str(id)].values())) for id in ids]) for v in vis[vi]])
            )

In [9]:
# for task in tasks_dv1:
#     res = scipy.stats.linregress(post_processed_ratings[task], post_processed_dv1[task])
#     plt.scatter(post_processed_ratings[task], post_processed_dv1[task])
#     plt.plot(range(1, 6), res.intercept + res.slope*range(1, 6), 'orange')
#     if task == "Navigate":
#         plt.title(f"[{task}] collision time ratio vs. subject rating (corr={np.round(res.rvalue, decimals=3)}, p={np.round(res.pvalue, decimals=3)})")
#     plt.title(f"[{task}] {tasks_dv1[task]} vs. subject rating (corr={np.round(res.rvalue, decimals=3)}, p={np.round(res.pvalue, decimals=3)})")
#     plt.xlabel("Subject rating")
#     plt.rcParams["figure.figsize"] = (6, 4)
#     plt.show()

## Investigating subject rating and 2nd DVs

In [10]:
dv2_data.keys()

dict_keys(['mean_pupil_dilation', 'total_body_translation', 'total_body_rotation', 'total_fixation_count', 'total_fixation_duration', 'total_gaze_movement', 'total_hmd_rotation', 'total_hmd_translation', 'total_left_hand_rotation', 'total_left_hand_translation', 'total_right_hand_rotation', 'total_right_hand_translation'])

In [11]:
tasks_dv2 = {
    "Slice": "total_right_hand_translation",
    "Throw": "total_right_hand_translation",
    "Catch": "total_body_translation",
    "Navigate": "total_hmd_rotation",
    "Place": "total_right_hand_translation",    
    "Wipe": "total_body_translation"
}

post_processed_dv2 = {task: [] for task in tasks_dv2}

for task in tasks_dv2:
    for vi in vis:
        post_processed_dv2[task].append(np.mean(
            [np.mean([np.mean(
                np.array(list(dv2_data[tasks_dv2[task]][f"{task.lower()}_{v}"][str(id)].values())) / np.array(list(dv1_data["task_completion_time"][f"{task.lower()}_{v}"][str(id)].values()))
            ) for id in ids])
            for v in vis[vi][-1:]])
        )

In [12]:
# for task in tasks_dv2:
#     res = scipy.stats.linregress(post_processed_ratings[task], post_processed_dv2[task])
#     plt.scatter(post_processed_ratings[task], post_processed_dv2[task])
#     plt.plot(range(1, 6), res.intercept + res.slope*range(1, 6), 'orange')
#     plt.title(f"[{task}] {tasks_dv2[task]}_ratio vs. subject rating (corr={np.round(res.rvalue, decimals=3)}, p={np.round(res.pvalue, decimals=3)})")
#     plt.xlabel("Subject rating")
#     plt.rcParams["figure.figsize"] = (6, 4)
#     plt.show()

### Multivariate Regression

In [13]:
standardize = True
normalize = lambda x, y: x / y
average = False
outlier_removal = False

In [14]:
redv = {}
for dv in [
    "total_gaze_movement",
    "total_body_translation",
    "total_body_rotation",
    "total_hmd_translation",
    "total_hmd_rotation",
    "total_right_hand_translation",
    "total_right_hand_rotation",
    "total_left_hand_translation",
    "total_left_hand_rotation",
]:
    redv[dv] = {}
    for task in tasks:
        redv[dv][task] = {}
        for vi in ori_vis:
            redv[dv][task][vi] = {}
            for id in ids:
                redv[dv][task][vi][id] = np.array(list(dv2_data[dv][f"{task.lower()}_{vi}"][str(id)].values())) / np.array(list(dv1_data["task_completion_time"][f"{task.lower()}_{vi}"][str(id)].values()))

# normalize
for dv in redv:
    for task in tasks:
        for vi in ori_vis:
            for id in ids:
                redv[dv][task][vi][id] = normalize(redv[dv][task][vi][id], np.mean(redv[dv][task]["normal_1"][id]))
                # average
                if average:
                    redv[dv][task][vi][id] = [np.mean(redv[dv][task][vi][id])]

# standardize
if standardize:
    for dv in redv:
        for task in tasks:
            for vi in ori_vis:
                mean, std = np.mean([redv[dv][task][vi][id] for id in ids]), np.std([redv[dv][task][vi][id] for id in ids])
                for id in ids:
                    redv[dv][task][vi][id] -= mean
                    if std:
                        redv[dv][task][vi][id] /= std

In [15]:
tasks_re = ["Place", "Wipe", "Slice", "Navigate", "Throw", "Catch"]
reiv = {}
for task in tasks_re:
    reiv[tasks_dv1[task]] = {}
for task in tasks_re:
    for vi in ori_vis:
        reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"] = {}
        for id in ids:
            if task == "Throw":
                reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] = -normalize(
                    np.array(list(dv1_data[tasks_dv1[task]][f"{task.lower()}_{vi}"][str(id)].values())), 
                    dv1_data["throw_failure_rate"][f"{task.lower()}_normal_1"][str(id)]['0']
                )
            else:
                reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] = normalize(
                    np.array(list(dv1_data[tasks_dv1[task]][f"{task.lower()}_{vi}"][str(id)].values())), 
                    np.mean(list(dv1_data[tasks_dv1[task]][f"{task.lower()}_normal_1"][str(id)].values()))
                )
            # average
            if average:
                reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] = [np.mean(reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id])]

In [16]:
models = {task: {vi: {} for vi in ori_vis} for task in tasks_re}
rsq = []
for task in tasks_re:
    for vi in ori_vis[:-1]:
        x, groups = [], []
        for id in ids:
            for trial in range(len(redv[dv][task][vi][id])):
                if task == "Catch" or average:
                    x.append([redv[dv][task][vi][id][trial] for dv in list(redv.keys())[:-2]])
                else:
                    x.append([redv[dv][task][vi][id][trial] for dv in redv])
            groups.append([id] * len(redv[dv][task][vi][id]))
        y = np.array([reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] for id in ids]).flatten()
        x = np.array(x)
        x = add_constant(x)
        groups = np.array(groups).flatten()
        # outlier removal
        if outlier_removal:
            mask = np.abs(y - np.mean(y)) < 2 * np.std(y)
            x = x[mask]
            y = y[mask]
            groups = groups[mask]
        models[task][vi] = OLS(y, x).fit()
        rsq.append(models[task][vi].rsquared_adj)
        print(f"{task}_{vi}", models[task][vi].summary(), "\n" * 10)
print(np.mean(rsq))

Place_cataract_1                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     2.919
Date:                Wed, 01 Feb 2023   Prob (F-statistic):             0.0553
Time:                        15:32:33   Log-Likelihood:                -4.5520
No. Observations:                  20   AIC:                             29.10
Df Residuals:                      10   BIC:                             39.06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3306      0.096   

c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\scipy\stats\_stats_py.py

### Aggregate across VI categories

In [17]:
rsq = []
models = {task: None for task in tasks_re}
for task in tasks_re:
    for v in vis:
        x, y, groups = [], [], []
        for vi in vis[v]:
            for id in ids:
                for trial in range(len(redv[dv][task][vi][id])):
                    x.append([redv[dv][task][vi][id][trial] for dv in redv])
                groups.extend([id] * len(redv[dv][task][vi][id]))
        y = np.hstack([np.array([reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] for id in ids]).flatten() for vi in vis[v]])
        x = np.array(x)
        x = add_constant(x)
        groups = np.array(groups).flatten()
        # outlier removal
        if outlier_removal:
            mask = np.abs(y - np.mean(y)) < 2 * np.std(y)
            x = x[mask]
            y = y[mask]
            groups = groups[mask]
        models[task] = OLS(y, x, missing='drop').fit()
        rsq.append(models[task].rsquared_adj)
        print(f"{task}_{v}", models[task].summary(), "\n" * 10)
print(np.mean(rsq))

Place_Glaucoma                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                 -0.087
Method:                 Least Squares   F-statistic:                    0.4745
Date:                Wed, 01 Feb 2023   Prob (F-statistic):              0.885
Time:                        15:32:34   Log-Likelihood:                -109.50
No. Observations:                  60   AIC:                             239.0
Df Residuals:                      50   BIC:                             259.9
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0503      0.212     

c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\statsmodels\regression\linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\statsmodels\regression\linear_model.py:940: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
c:\Users\capri28\mambaforge\envs\igvi\lib\site-packages\statsmodels\stats\stattools.py:50: RuntimeWarning: invalid value encountered in double_scalars

### Aggregate across VIs

In [18]:
rsq = []
models = {task: None for task in tasks_re}
for task in tasks_re:
    x, y, groups = [], [], []
    for vi in ori_vis[:-1]:
        for id in ids:
            for trial in range(len(redv[dv][task][vi][id])):
                x.append([redv[dv][task][vi][id][trial] for dv in redv])
            groups.extend([id] * len(redv[dv][task][vi][id]))
    y = np.hstack([np.array([reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] for id in ids]).flatten() for vi in ori_vis[:-1]])
    x = np.array(x)
    x = add_constant(x)
    groups = np.array(groups).flatten()
    # outlier removal
    if outlier_removal:
        mask = np.abs(y - np.mean(y)) < 2 * np.std(y)
        x = x[mask]
        y = y[mask]
        groups = groups[mask]
    models[task] = OLS(y, x, missing='drop').fit()
    rsq.append(models[task].rsquared_adj)
    print(task, models[task].summary(), "\n" * 10)
print(np.mean(rsq))

Place                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.6727
Date:                Wed, 01 Feb 2023   Prob (F-statistic):              0.734
Time:                        15:32:34   Log-Likelihood:                -564.46
No. Observations:                 300   AIC:                             1149.
Df Residuals:                     290   BIC:                             1186.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9341      0.093     20.739   

### Aggregate across tasks

In [22]:
task_weights = {
    "Place": 28/2,
    "Wipe": 28/5,
    "Slice": 28/5,
    "Navigate": 28/5,
    "Throw": 28/10,
    "Catch": 28/1,
}
models = {vi: None for vi in vis}
rsq = []
for vi in ori_vis[:-1]:
    x, y, groups, weights = [], [], [], []
    for task in tasks_re:
        for id in ids:
            for trial in range(len(redv[dv][task][vi][id])):
                x.append([redv[dv][task][vi][id][trial] for dv in redv])
            groups.extend([id] * len(redv[dv][task][vi][id]))
            weights.extend([task_weights[task]] * len(redv[dv][task][vi][id]))
    y = np.hstack([np.array([reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] for id in ids]).flatten() for task in tasks_re])
    x = np.array(x)
    x = add_constant(x)
    groups = np.array(groups).flatten()
    weights = np.array(weights).flatten()
    # outlier removal
    if outlier_removal:
        mask = np.abs(y - np.mean(y)) < 2 * np.std(y)
        x = x[mask]
        y = y[mask]
        groups = groups[mask]
        weights = np.array(weights)[mask]
    models[vi] = WLS(y, x, weights, missing='drop').fit()
    rsq.append(models[vi].rsquared_adj)
    print(vi, models[vi].summary(), "\n" * 10)
print(np.mean(rsq))

cataract_1                             WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.045
Model:                            WLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.406
Date:                Wed, 01 Feb 2023   Prob (F-statistic):              0.185
Time:                        16:54:29   Log-Likelihood:                -623.70
No. Observations:                 280   AIC:                             1267.
Df Residuals:                     270   BIC:                             1304.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1348      0.124      9.1

### Aggregate across all

In [21]:
rsq = []
x, y, groups, weights = [], [], [], []
for vi in ori_vis[:-1]:
    for task in tasks_re:
        for id in ids:
            for trial in range(len(redv[dv][task][vi][id])):
                x.append([redv[dv][task][vi][id][trial] for dv in redv])
            groups.extend([id] * len(redv[dv][task][vi][id]))
            weights.extend([task_weights[task]] * len(redv[dv][task][vi][id]))
    y.append(np.hstack([np.array([reiv[tasks_dv1[task]][f"{task.lower()}_{vi}"][id] for id in ids]).flatten() for task in tasks_re]))
x = np.array(x)
x = add_constant(x)
y = np.array(y).flatten()
groups = np.array(groups).flatten()
weights = np.array(weights).flatten()
# outlier removal
if outlier_removal:
    mask = np.abs(y - np.mean(y)) < 2 * np.std(y)
    x = x[mask]
    y = y[mask]
    groups = groups[mask]
    weights = np.array(weights)[mask]
model = WLS(y, x, weights, missing='drop').fit()
print("All", model.summary())

All                             WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            WLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     3.210
Date:                Wed, 01 Feb 2023   Prob (F-statistic):           0.000696
Time:                        16:54:20   Log-Likelihood:                -14239.
No. Observations:                4200   AIC:                         2.850e+04
Df Residuals:                    4190   BIC:                         2.856e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9255      0.101     19.122     